In [3]:
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [1]:
import tweetnlp
import numpy as np
import pandas as pd
df_all_tweets = pd.read_csv('../team_data/all_tweets.csv', index_col=0)

In [2]:
model_nlp = tweetnlp.load_model('sentiment')  # Or `model = tweetnlp.Sentiment()` 
# model_nlp.sentiment("Yes, including Medicare and social security saving👍", return_probability=True)  # Or `model.predict`

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
df_sample = df_all_tweets.iloc[:1000]
df_sample.tail()

,Unnamed: 0,week,team,timestamp,tweet_text,userid
995,995,12,49ers,2022-11-26 19:51:00+00:00,NFL: San Francisco 49ers at Carolina Panthers ...,3017960336
996,996,12,49ers,2022-11-26 19:48:53+00:00,Huge W for this game worn jersey fusion swatch...,1484398102849245184
997,997,12,49ers,2022-11-26 19:48:34+00:00,@Kay_Breezy22 Deebo continues disappointing fa...,1596153271361720321
998,998,12,49ers,2022-11-26 19:46:00+00:00,Troubling Details Emerge From Deebo Samuel's L...,3017960336
999,999,12,49ers,2022-11-26 19:40:04+00:00,Need someone to make this with or with out the...,2375943984


In [28]:
import numpy as np
def get_labels(results):
  labels = []
  for result in results:
    label = result['label']
    score = result['probability'][result['label']]
    labels.append({'label': label, 'score': score})
  return labels

def sample_sentiment(df):
  tweets = df['tweet_text'].to_list()
  results = model_nlp.sentiment(tweets, return_probability=True)
  labels = get_labels(results)
  sentiment_df = pd.DataFrame.from_dict(labels)
  # concat sentiment df with tweet df and return
  df_tweets_w_sentiment = pd.concat([df, sentiment_df], axis=1)
  return df_tweets_w_sentiment
  
def sentiment_batches(df, batches):
  split_dfs = np.array_split(df_all_tweets, batches)
  print(len(split_dfs))
  for i, df in enumerate(split_dfs):
    split_dfs[i] = [i,df]
  
  # for row in split_dfs:
  #   print(row[0])
  #   df = row[1]
  #   sent_df = sample_sentiment(df)
  #   split_dfs[row[0]][1] = sent_df
  return split_dfs

sent_df_sample = sentiment_batches(df_sample,10)

10


In [32]:
sent_df_sample[9][1]

,Unnamed: 0,week,team,timestamp,tweet_text,userid
580556,580556,19,49ers,2023-01-14 14:42:25+00:00,Let’s go #FTTB !! https://t.co/aKXnf7ysNe,953081648500715520
580557,580557,19,49ers,2023-01-14 14:42:05+00:00,"@crystalscuor Crys, it’s nasty outside here. I...",4885088052
580558,580558,19,49ers,2023-01-14 14:41:59+00:00,GAME DAY 🔑 #FTTB https://t.co/X23BQHBcLy,15068717
580559,580559,19,49ers,2023-01-14 14:41:42+00:00,Time for CMC to run wild in the wild card roun...,1166193243123593216
580560,580560,19,49ers,2023-01-14 14:41:01+00:00,"Brock Purdy, Skylar Thompson are lowest-drafte...",3017960336
...,...,...,...,...,...,...
645057,645057,19,vikings,2023-01-11 20:33:58+00:00,@heykayadams @UpAndAdamsShow @kgxix @Young_Sla...,1422946127750606853
645058,645058,19,vikings,2023-01-11 20:32:47+00:00,Here’s the first #Giants injury report for the...,1561574120
645059,645059,19,vikings,2023-01-11 20:32:00+00:00,22 Things the 2022 Regular Season Revealed Abo...,3035196396
645060,645060,19,vikings,2023-01-11 20:31:00+00:00,Cook on The Running Game Heading Into Sunday: ...,3035196396


1,000 rows: 200 seconds
5,000 rows: died


In [21]:
def get_label(result):
    label = result['label']
    score = result['probability'][result['label']]
    return({'label': label, 'score': score})


def get_single_sentiment(text):
    result = model_nlp.sentiment(text, return_probability=True)
    return get_label(result)

def sample_sentiment_iterate(df_all, n):
    df_sample = df_all.sample(n=n, random_state=15)
    tweets = df_sample['tweet_text'].to_list()
    labels = []
    for tweet in tweets:
        label = get_single_sentiment(tweet)
        # print(label)
        labels.append(label)
    return labels


single_labels = sample_sentiment_iterate(df_all_tweets, 500)

500 rows = 44 seconds

In [17]:
single_labels

[{'label': 'positive', 'score': 0.8440860509872437},
 {'label': 'positive', 'score': 0.6936076879501343},
 {'label': 'positive', 'score': 0.6192249655723572},
 {'label': 'neutral', 'score': 0.699512779712677},
 {'label': 'neutral', 'score': 0.502205491065979},
 {'label': 'positive', 'score': 0.9252962470054626},
 {'label': 'positive', 'score': 0.9453657269477844},
 {'label': 'positive', 'score': 0.9387558698654175},
 {'label': 'positive', 'score': 0.6105028986930847},
 {'label': 'negative', 'score': 0.43416789174079895}]

In [22]:
sample_sentiment(df_all_tweets, 500)

[{'label': 'positive', 'score': 0.844086229801178},
 {'label': 'positive', 'score': 0.6936076879501343},
 {'label': 'positive', 'score': 0.6192253232002258},
 {'label': 'neutral', 'score': 0.6995124816894531},
 {'label': 'neutral', 'score': 0.5022056698799133},
 {'label': 'positive', 'score': 0.9252962470054626},
 {'label': 'positive', 'score': 0.9453657269477844},
 {'label': 'positive', 'score': 0.9387557506561279},
 {'label': 'positive', 'score': 0.6105028390884399},
 {'label': 'negative', 'score': 0.4341687858104706},
 {'label': 'positive', 'score': 0.8647853136062622},
 {'label': 'negative', 'score': 0.5846498608589172},
 {'label': 'neutral', 'score': 0.8953248858451843},
 {'label': 'neutral', 'score': 0.7397492527961731},
 {'label': 'positive', 'score': 0.9883453845977783},
 {'label': 'neutral', 'score': 0.9394051432609558},
 {'label': 'positive', 'score': 0.9664994478225708},
 {'label': 'neutral', 'score': 0.4657299518585205},
 {'label': 'positive', 'score': 0.7178213000297546},
